## Set Up
- access to HPC system of CAU
- set up conda environments for the bioinformatic pipelines


# Installation of SSH client
Bitvise SSH Client
Host: nesh-login.rz.uni-kiel.de
Port: 22
Username: smomw681
Initial method: password
- Enter password and log in 
-> access to SFTP and terminal directly connected the HPC system

Acoount_login: 

$WORK quota set to 10 TB (hard), 9 TB (soft) 

Absolute path to my storage: /gxfs_work/geomar/smomw681


# Set up of required environment
The conda package is already installed in the working directory of the smomw681. 
Conda environments for each of the pipeline are required, containing following bioconda, other packages and modules:




In [ ]:
# For each and every enviroments: 
    module load gcc12-env/12.3.0
    module load miniconda3/24.11.1

# Preprocessing: 
    conda create --name FastqDump 
    conda activate FastqDump
    # basic modules loaded
    conda install bioconda::sra-tools -y
    conda install bioconda::parallel-fastq-dump -y 

# Assembly
    conda create --name Assembly
    conda activate Assembly
    conda install bioconda::spades -y

    
- AntiSMASH
- Deep ARG